In [1]:
!pip install koalas

In [1]:
import pandas as pd
import numpy as np
import databricks.koalas as ks

import argparse
from collections import defaultdict
import datetime
from pprint import pprint
from google.cloud import datastore
import google.cloud.exceptions

from databricks.koalas.config import option_context

from databricks.koalas.config import set_option, get_option


from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

In [2]:
ks.get_option('compute.max_rows')

1000

In [3]:
ks.set_option('compute.max_rows', 100000)
ks.get_option('compute.max_rows')

100000

In [4]:
ks.get_option('compute.shortcut_limit')

1000

In [5]:
ks.set_option('compute.shortcut_limit', 1000000)
ks.get_option('compute.shortcut_limit')

1000000

In [6]:
client = datastore.Client()

## Edges Creation

In [7]:
edges = pd.read_csv("/tmp/links.csv")

In [8]:
kdf_edges = ks.from_pandas(edges)


In [9]:
kdf_edges.head()

,sender,receiver,sender_cds_caut,receiver_cds_caut,sender_cds_csem,receiver_cds_csem,sender_csem_exit,receiver_csem_exit,first_datetime,last_datetime,count_of_sending,count_of_receiving,amount_of_sending,amount_of_receiving
0,0x0000000048246c5f37df029bef0cefd9ee36f301,0xe87729bcf07cd2be7333a393b94c0959e30d9f96,0,0,0,1,NaN,NaN,5/15/2018 9:00,1/15/2019 9:00,7,0,0.100490,0.0
1,0x00004225550a3942d1877a3a43df923eb5279a5b,0xb3aaaae47070264f3595c5032ee94b620a583a39,0,0,1,0,NaN,NaN,5/15/2018 9:00,1/15/2019 9:00,7,0,0.347569,0.0
2,0x00008e23e286645c9c1e13ec3b02e18d0ddd1d7d,0xe1134bfd57131ce3ecfe69300b39728fd5c51573,0,0,0,0,NaN,NaN,5/15/2018 9:00,1/15/2019 9:00,7,0,2.297845,0.0
3,0x0001f2fe9bc9e12b975d575f67321121e2579b6d,Yobit,0,0,0,0,NaN,NaN,5/15/2018 9:00,1/15/2019 9:00,7,0,0.342718,0.0
4,0x00029638d51bd8ef7da7f43911aa9aaafe28dc7e,0xdc25e92d0f361f54052a1a4ad934b9184f13b255,0,0,1,0,NaN,NaN,5/15/2018 9:00,1/15/2019 9:00,7,0,0.106384,0.0


In [10]:
kdf_edges.info()

<class 'databricks.koalas.frame.DataFrame'>
Index: 219806 entries, 54272 to 164863
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   sender               219806 non-null  object 
 1   receiver             219806 non-null  object 
 2   sender_cds_caut      219806 non-null  int64  
 3   receiver_cds_caut    219806 non-null  int64  
 4   sender_cds_csem      219806 non-null  int64  
 5   receiver_cds_csem    219806 non-null  int64  
 6   sender_csem_exit     0 non-null       float64
 7   receiver_csem_exit   0 non-null       float64
 8   first_datetime       219806 non-null  object 
 9   last_datetime        219806 non-null  object 
 10  count_of_sending     219806 non-null  int64  
 11  count_of_receiving   219806 non-null  int64  
 12  amount_of_sending    219806 non-null  float64
 13  amount_of_receiving  219806 non-null  float64
dtypes: float64(4), int64(6), object(4)

In [12]:
test = ["test1","test2"]
"test1" not in test

False

In [52]:
def gnode_edge_update(sender, receiver):   
#     print("calling it first")

#     client = datastore.Client()
    key = client.key("node",sender)
    task = client.get(key)

#     print(task)
#     if "edges" in task:
#         edges = task["edges"]
        
#         if receiver not in edges:
#             edges.append(receiver)
            
#         else:
#             return True
            
#     else:
#         edges = [receiver]

#     task["edges"] = edges

#     client.put(task)



    if receiver not in edges:
        edges.append(receiver)
        
        task["edges"] = edges

        client.put(task)

    

In [56]:
def gedge_create(row):
    
#     client = datastore.Client()
    gnode_edge_update(row["sender"], row["receiver"])
    gnode_edge_update(row["receiver"], row["sender"])
    
#     key = client.key("edges","{}-{}-{}".format(row["sender"],row["receiver"],row["last_datetime"]))
#     task = datastore.Entity(key)
#     task.update(
#         {
#             "sender":row["sender"],
#             "receiver":row["receiver"],
#             "sender_cds_caut":row["sender_cds_caut"],
#             "receiver_cds_caut":row["receiver_cds_caut"],
#             "sender_cds_csem":row["sender_cds_csem"],
#             "receiver_cds_csem":row["receiver_cds_csem"],
#             "sender_csem_exit":row["sender_csem_exit"],
#             "receiver_csem_exit":row["receiver_csem_exit"],
#             "first_datetime":row["first_datetime"],
#             "last_datetime":row["last_datetime"],
#             "count_of_sending":row["count_of_sending"],
#             "count_of_receiving":row["count_of_receiving"],
#             "amount_of_sending":row["amount_of_sending"],
#             "amount_of_receiving":row["amount_of_receiving"]
#         }
#     )
#     client.put(task)
    
    global counter 
    
    counter += 1
    
    if counter % 1000 == 0 or counter == 26805 :
        print("Progress @ {}% , {}".format(round(counter/26805*100), counter))
        

In [ ]:
%%time
counter = 0
# with option_context('compute.max_rows', 10000000000): 
#     kdf_nodes.apply(gnode_create,axis='columns')
try:
    kdf_edges = ks.from_pandas(edges)
    kdf_edges.apply(gedge_create,axis='columns')

except ValueError:
    print("Value error with counter @ {}".format(counter))
    

Progress @ 4% , 1000
Progress @ 7% , 2000
Progress @ 11% , 3000
Progress @ 15% , 4000
Progress @ 19% , 5000
Progress @ 22% , 6000
Progress @ 26% , 7000
Progress @ 30% , 8000
Progress @ 34% , 9000
Progress @ 37% , 10000
Progress @ 41% , 11000
Progress @ 45% , 12000
Progress @ 48% , 13000
Progress @ 52% , 14000
Progress @ 56% , 15000
Progress @ 60% , 16000
Progress @ 63% , 17000
Progress @ 67% , 18000
Progress @ 71% , 19000
Progress @ 75% , 20000
Progress @ 78% , 21000
Progress @ 82% , 22000
Progress @ 86% , 23000
Progress @ 90% , 24000
Progress @ 93% , 25000
Progress @ 97% , 26000
Progress @ 100% , 26805
Value error with counter @ 26805
CPU times: user 1h 33min 26s, sys: 47 s, total: 1h 34min 13s
Wall time: 3h 43min 19s


In [27]:
df_edges.printSchema()

root
 |-- sender: string (nullable = true)
 |-- receiver: string (nullable = true)
 |-- sender_cds_caut: long (nullable = true)
 |-- receiver_cds_caut: long (nullable = true)
 |-- sender_cds_csem: long (nullable = true)
 |-- receiver_cds_csem: long (nullable = true)
 |-- sender_csem_exit: double (nullable = true)
 |-- receiver_csem_exit: double (nullable = true)
 |-- first_datetime: string (nullable = true)
 |-- last_datetime: string (nullable = true)
 |-- count_of_sending: long (nullable = true)
 |-- count_of_receiving: long (nullable = true)
 |-- amount_of_sending: double (nullable = true)
 |-- amount_of_receiving: double (nullable = true)



In [35]:
@pandas_udf("string")
def gedge_create2(sender: pd.Series, receiver: pd.Series) -> pd.Series:
    
    gnode_edge_update(sender, receiver)
    gnode_edge_update(receiver, sender)
    
    
    return sender

In [ ]:
df_edges.select(gedge_create2("sender","receiver")).collect()

In [ ]:
%%time
counter = 0

kdf_edges.apply(gedge_create,axis='columns')


In [22]:
df_edges = spark.createDataFrame(edges[0:9999])

In [23]:
df_edges.printSchema()

root
 |-- sender: string (nullable = true)
 |-- receiver: string (nullable = true)
 |-- sender_cds_caut: long (nullable = true)
 |-- receiver_cds_caut: long (nullable = true)
 |-- sender_cds_csem: long (nullable = true)
 |-- receiver_cds_csem: long (nullable = true)
 |-- sender_csem_exit: double (nullable = true)
 |-- receiver_csem_exit: double (nullable = true)
 |-- first_datetime: string (nullable = true)
 |-- last_datetime: string (nullable = true)
 |-- count_of_sending: long (nullable = true)
 |-- count_of_receiving: long (nullable = true)
 |-- amount_of_sending: double (nullable = true)
 |-- amount_of_receiving: double (nullable = true)



In [ ]:
%%time
counter = 0
df_edges.groupby("sender").applyInPandas(gedge_create,schema="sender string,\
receiver string,\
sender_cds_caut long,\
receiver_cds_caut long,\
sender_cds_csem long,\
receiver_cds_csem long,\
sender_csem_exit double,\
receiver_csem_exit double,\
first_datetime string,\
last_datetime string,\
count_of_sending long,\
count_of_receiving long,\
amount_of_sending double,\
amount_of_receiving double, col1 string").collect()

## Nodes creation process

In [13]:
nodes = pd.read_csv("/tmp/nodes.csv")

In [14]:
kdf_nodes = ks.from_pandas(nodes)

In [15]:
kdf_nodes.head()

,cid,first_datetime,last_datetime,count_of_sending,amount_of_sending,count_of_receiving,amount_of_receiving,count_of_rbwm_csem_connections,count_of_non_rbwm_csem_connections,count_of_rbwm_caut_connections,count_of_non_rbwm_caut_connections,count_of_total_connections,join_date,ip_address_array,email,home_tel_number,mobile_pager_number
0,0x0000000048246c5f37df029bef0cefd9ee36f301,1/15/2019 9:00,1/15/2019 9:00,7,0.100490,0,0.000000,1,0,0,0,1,3/27/2008 0:00,"[""241.92.245.185"",""188.22.55.210"",""58.215.140....",8526@gmail.com,2092,4882
1,0x000000988c55d7aab7ccfda5bb273b7294fcbac5,1/15/2019 9:00,1/15/2019 9:00,0,0.000000,7,0.011004,0,1,0,1,1,6/22/1995 0:00,"[""113.141.115.152"",""120.228.13.147"",""191.29.83...",8136@gmail.com,6033,5758
2,0x00004225550a3942d1877a3a43df923eb5279a5b,1/15/2019 9:00,1/15/2019 9:00,7,0.347569,7,0.350509,0,1,0,0,2,6/14/2015 0:00,"[""160.88.215.46"",""187.240.128.176"",""195.224.24...",691@gmail.com,6183,8840
3,0x00008e23e286645c9c1e13ec3b02e18d0ddd1d7d,1/15/2019 9:00,1/15/2019 9:00,7,2.297845,7,2.299315,0,1,0,0,2,9/30/2016 0:00,"[""230.0.74.119"",""98.95.183.118"",""242.171.131.1...",5654@gmail.com,1749,1785
4,0x000130c1447fe71a395b48952b545b1760f37f8e,1/15/2019 9:00,1/15/2019 9:00,0,0.000000,7,0.537672,0,0,0,0,1,1/21/2018 0:00,"[""115.221.252.86"",""175.130.126.242"",""45.126.23...",3679@gmail.com,3041,4167


In [16]:
kdf_nodes.info()

<class 'databricks.koalas.frame.DataFrame'>
Index: 194217 entries, 0 to 194216
Data columns (total 17 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   cid                                 194217 non-null  object 
 1   first_datetime                      194217 non-null  object 
 2   last_datetime                       194217 non-null  object 
 3   count_of_sending                    194217 non-null  int64  
 4   amount_of_sending                   194217 non-null  float64
 5   count_of_receiving                  194217 non-null  int64  
 6   amount_of_receiving                 194217 non-null  float64
 7   count_of_rbwm_csem_connections      194217 non-null  int64  
 8   count_of_non_rbwm_csem_connections  194217 non-null  int64  
 9   count_of_rbwm_caut_connections      194217 non-null  int64  
 10  count_of_non_rbwm_caut_connections  194217 non-null  int64  
 11  count_of_total_connection

In [17]:
def gnode_create(row):
    key = client.key("node",row["cid"])
    task = datastore.Entity(key)
    task.update(
        {
           "first_datetime":row["first_datetime"],
            "last_datetime":row["last_datetime"],
            "count_of_sending":row["count_of_sending"],
            "amount_of_sending":row["amount_of_sending"],
            "count_of_receiving":row["count_of_receiving"],
            "amount_of_receiving":row["amount_of_receiving"],
            "count_of_rbwm_csem_connections":row["count_of_rbwm_csem_connections"],
            "count_of_non_rbwm_csem_connections":row["count_of_non_rbwm_csem_connections"],
            "count_of_rbwm_caut_connections":row["count_of_rbwm_caut_connections"],
            "count_of_non_rbwm_caut_connections":row["count_of_non_rbwm_caut_connections"],
            "count_of_total_connections":row["count_of_total_connections"],
            "join_date":row["join_date"],
            "ip_address_array":row["ip_address_array"],
            "email":row["email"],
            "home_tel_number":row["home_tel_number"],
            "mobile_pager_number":row["mobile_pager_number"],
            "edges":[]
        }
    )
    client.put(task)
    
    global counter 
    
    counter += 1
    
    if counter % 10000 == 0 or counter == 194217:
        print("Progress @ {}% , {}".format(round(counter/194217*100), counter))

In [ ]:
%%time
counter = 0
# with option_context('compute.max_rows', 10000000000): 
#     kdf_nodes.apply(gnode_create,axis='columns')
try:
    kdf_nodes.apply(gnode_create,axis='columns')

except ValueError:
    print("Value error with counter @ {}".format(counter))
    


KeyboardInterrupt: 